<a href="https://colab.research.google.com/github/kaorisugi/diveintocode-ml/blob/master/sprint19_segmentation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# sprint19 セグメンテーション１


KaggleのTGS Salt Identification Challengeのデータセットを使用し、セグメンテーションを行います。

TGS Salt Identification Challenge | Kaggle

セグメンテーション手法としてU-Net[1]を使います。

[1]O.Ronneberger, P.Fischer, T.Brox. U-Net: Convolutional Networks for Biomedical Image Segmentation. Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234–241, 2015

https://arxiv.org/pdf/1505.04597.pdf

### 【問題1】学習・推定
以下のKeras実装を使用して学習・推定を行ってください。

zhixuhao/unet: unet for image segmentation

《GPU環境での学習》

大規模なデータセット、大きなモデルになるため、GPUを使用する必要があります。

《新たなデータセットの適用》

公開されている実装で用意されたものとは異なるデータセットを入力するための準備が必要です。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


####GItHubのUnetのデータをクローンする

In [0]:
#Unet用のディレクトリを作成
%cd /content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet

/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet


In [0]:
!git clone "https://github.com/zhixuhao/unet.git"

Cloning into 'unet'...
remote: Enumerating objects: 394, done.
remote: Total 394 (delta 0), reused 0 (delta 0), pack-reused 394
Receiving objects: 100% (394/394), 44.91 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (80/80), done.
Checking out files: 100% (253/253), done.


In [0]:
#GitHubのUnetのデータをそのまま使ってunetを実行してみる
#gitHubのUnetを実行(!cdでUnetのフォルダに移動しておく)
! python '/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/main.py'

Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/1
Found 30 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
2019-09-23 03:43:40.073413: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-09-23 03:43:40.073642: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1e7ca00 executing computations on platform Host. Devices:
2019-09-23 03:43:40.073673: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-09-23 03:43:40.076281: I te

タスク
*   kaggleのデータセットをダウンロード
*   実装したコードのパスを書き換える
*   論文を読んだ上で、実装したコードのコードリーディングをする


####kaggleの塩コンペのデータセットをダウンロードする


In [0]:
!mkdir .kaggle

In [0]:
!ls -a

data		   .git     LICENSE   __pycache__      unet_membrane.hdf5
dataPrepare.ipynb  img	    main.py   README.md
data.py		   .kaggle  model.py  trainUnet.ipynb


In [0]:
#所有者のみが使える設定
!chmod 600 '/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/.kaggle/kaggle.json'

In [0]:
import json

token = {'username':'kaorisugi','key':'3652816250535d833485a35952069a51'}
with open('/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp .kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c tgs-salt-identification-challenge

  0% 0.00/322k [00:00<?, ?B/s]
100% 322k/322k [00:00<00:00, 24.4MB/s]
  0% 0.00/264k [00:00<?, ?B/s]
100% 264k/264k [00:00<00:00, 16.7MB/s]
  0% 0.00/922k [00:00<?, ?B/s]
100% 922k/922k [00:00<00:00, 61.0MB/s]
 94% 154M/163M [00:02<00:00, 53.0MB/s]
100% 163M/163M [00:02<00:00, 59.6MB/s]
 76% 29.0M/37.9M [00:00<00:00, 23.2MB/s]
100% 37.9M/37.9M [00:00<00:00, 51.9MB/s]


In [0]:
!unzip -d test train.zip

In [0]:
!unzip -d test test.zip

#### kaggleデータセットの入力
解凍したkaggleのtrain testデータを、Unet内のコードのPathのフォルダに入れて学習、推定できるようにする（コマンドを使わずgoogle driveで作業してしまいました）

In [0]:
#ローカルで書き換えた.pyファイルを移動
!mv "/content/data.py" "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet"
!mv "/content/main.py" "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet"
!mv "/content/model.py" "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet"

mv: cannot stat '/content/data.py': No such file or directory
mv: cannot stat '/content/main.py': No such file or directory


In [0]:
#unetフォルダに移動して、main.pyファイルを実行
%cd "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet"
!python "/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/main.py"

/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet
Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/content/drive/My Drive/Colab Notebooks/DIVE/kadai/sprint19/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/1
/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)
Found 5195 images belonging to 1 classes.
Found 4000 images belonging to 1 classes.
2019-09-24 02:47:21.959632: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-09-24 02:47:21.96413

##### 推定結果画像は添付しましたが、学習が足りなかったのか、グレーの画像でした

### 【問題2】コードリーディング

論文[1]に目を通した上で、上記実装のコードリーディングを行ってください。

# main.py

In [0]:
from model import *
from data import *

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#dictコンストラクタでキーワード引数を作成（下の関数trainGeneratorに渡す）
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

#data.py内の関数trainGeneratorをインスタンス化。
myGene = trainGenerator(2,'data/membrane/train','image','label',data_gen_args,save_to_dir = None)

#model.py内の関数unetをインスタンス化
model = unet()
#model.py内でインポートした関数ModelCheckpointをインスタンス化（エポックごとに、'unet_membrane.hdf5'に最良のモデルのみを保存）
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
#model.py内で呼び出したkeras.modelsからメソッドfit_generatorをインポートし、インスタンスmyGeneによってモデルを学習させる。戻り値はHistroryオブジェクト
model.fit_generator(myGene,steps_per_epoch=300,epochs=1,callbacks=[model_checkpoint])

#data.py内の関数testGeneratorをインスタンス化
testGene = testGenerator("data/membrane/test")
#model.py内で呼び出したkeras.modelからメソッドpredict_generatorをインポートし、インスタンス化。predict_generatorの戻り値は予測値の Numpy 配列
results = model.predict_generator(testGene,30,verbose=1)
#model.data.py内の関数saveResultを使って、上で定義したインスタンスresultsで生成した推定結果を指定のパスに保存する
saveResult("data/membrane/result",results)

# data.py

In [0]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import glob
import skimage.io as io
import skimage.transform as trans

Sky = [128,128,128]
Building = [128,0,0]
Pole = [192,192,128]
Road = [128,64,128]
Pavement = [60,40,222]
Tree = [128,128,0]
SignSymbol = [192,128,128]
Fence = [64,64,128]
Car = [64,0,128]
Pedestrian = [64,64,0]
Bicyclist = [0,128,192]
Unlabelled = [0,0,0]

#下で定義している関数labelVisualizeの引数
COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement,
                          Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])

#画像データを入力用数値に変換
def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):#flag_multi_class=Trueで、タスクが多値分類の場合
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):#1ピクセルについて、マスク内のクラスを見つけてone-Hotする
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):#flag_multi_class=Falseで、タスクが２値分類の場合
        img = img / 255
        mask = mask /255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)


#train用画像データを拡張し、テンソル形式に変換
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    #画像データの拡張（image,mask)メソッドをインスタンス化
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    #画像が保存されているパスを指定し、データ拡張かつテンソル形式に変換
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    #train,maskの要素をタプルにまとめたリストを取得（要素数はtrain,maskの少ない方）
    train_generator = zip(image_generator, mask_generator)
    #テンソル形式に変換
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)

#test画像を読み込んで、テンソル形式に変換
def testGenerator(test_path,num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    test_image_path_list = glob.glob(test_path + "/*")
    for i in range(num_image):
        #img = io.imread(os.path.join(test_path,"%d.png"%i),as_gray = as_gray)#io.imreadは画像を読み込んで配列にする
        img = io.imread(test_image_path_list[i], as_gray=as_gray)#kaggleのtest dataを読み込むように書き換え
        img = img / 255
        img = trans.resize(img,target_size)#画像のサイズを指定サイズに変更
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img #2値分類の場合(256,256,1),多値（256,256)
        img = np.reshape(img,(1,)+img.shape)#２値（1,256,256,1) 多値(1,256,256)
        yield img

#npyデータを作成したい場合に使う関数
def geneTrainNpy(image_path,mask_path,flag_multi_class = False,num_class = 2,image_prefix = "image",mask_prefix = "mask",image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = io.imread(item,as_gray = image_as_gray)
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr

#
def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img#imgが3次元の時は０番目のチャネルの（H,W）、それ以外（２次元）ならそのまま
    img_out = np.zeros(img.shape + (3,))#要素数３の次元を１つ足す（RGB配列が入る）
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255


#推定結果を保存する関数
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]#多値分類の場合
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)


# model.py

In [0]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler #各エポック後にモデルを保存する関数
from keras import backend as keras

#畳み込み層
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

#ここからアップサンプリング
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)#活性化関数

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model
